In [1]:
import numpy as np
import pandas as pd
from skdh.io import ReadCwa
from skdh.preprocessing import DETACH
from skdh import features
from skdh.sit2stand import Sit2Stand
from mobgap.utils.conversions import to_body_frame
from mobgap.gait_sequences import GsdIluz
from mobgap.initial_contacts import IcdShinImproved, refine_gs
from mobgap.laterality import LrcUllrich, strides_list_from_ic_lr_list
from mobgap.cadence import CadFromIc
from mobgap.stride_length import SlZijlstra
from mobgap.turning import TdElGohary
from mobgap.walking_speed import WsNaive
from mobgap.pipeline import GsIterator
from mobgap.utils.df_operations import create_multi_groupby
from mobgap.wba import StrideSelection, WbAssembly
from mobgap.aggregation import apply_thresholds, get_mobilised_dmo_thresholds, MobilisedAggregator
from mobgap.utils.interpolation import naive_sec_paras_to_regions
import warnings

# Load Catalog

In [2]:
catalog = pd.read_csv('./catalog.csv')

# SKDH

## Preprocessing

In [3]:
def wear_detection(data):
    detach = DETACH(increase_threshold=5)
    wear = detach.predict(
        time=data['time'],
        accel=data['accel'],
        temperature=data['temperature'],
        fs=data['fs']
    )
    start = wear['wear'][0,0]
    end = wear['wear'][0,1] - 100*60*60 # remove last hour
    return start, end # you can calculate the wearing duration with these

In [4]:
def slice_sample(data, start, end):
    sliced_data = {}
    for key, value in data.items():
        if key == 'fs':
            sliced_data[key] = value
        else:
            sliced_data[key] = value[start:end]
    return sliced_data

## Feature Extraction

In [5]:
sa_features = {
    # Signal Distribution
    'mean': features.Mean(), # The signal mean
    'standard_deviation': features.StdDev(), # The signal standard deviation
    'skewness': features.Skewness(), # The skewness of a signal
    'kurtosis': features.Kurtosis(), # The kurtosis of a signal
    'range': features.Range(), # The difference between the maximum and minimum value
    'interquartile_range': features.IQR(), # The difference between the 75th percentile and 25th percentile of the values
    'range_count_percentage': features.RangeCountPercentage(), # The percent of the signal that falls between specified values
    'ratio_beyond_r_sigma': features.RatioBeyondRSigma(), #The percent of the signal outside standard deviations from the mean
    # Time Domain
    'mean_cross_rate': features.MeanCrossRate(), # Number of signal mean value crossings
    'autocorrelation': features.Autocorrelation(), # The similarity in profile between the signal and a time shifted version of the signal
    'signal_entropy': features.SignalEntropy(), # A Measure of the information contained in a signal
    'jerk_metric': features.JerkMetric(), # The normalized sum of jerk
    'root_mean_square': features.RMS(), # The root mean square value of the signal
    'detail_power': features.DetailPower(), # The summed power in the detail levels that span the chosen frequency band
    # Frequency Domain
    'dominant_frequency': features.DominantFrequency(), # The primary frequency in the signal
    'power_spectral_sum': features.PowerSpectralSum(), # Sum of power spectral density values
    'power_range_sum': features.RangePowerSum(), # Sum of power spectral density values within the specified range
    'spectral_flatness': features.SpectralFlatness(), # A measure of the "tonality" or resonant structure of a signal
    'spectral_entropy': features.SpectralEntropy(), # A measure of the information contained in the power spectral density estimate
    'sparc': features.SPARC() # A quantitative measure of the smoothness of a signal
}

In [6]:
def get_features(data):
    bank = features.Bank()
    bank.add(sa_features.values())
    accel_features = bank.compute(data['accel'], fs=100, axis=0)
    accel_features = accel_features.reshape(1, -1)
    columns = [f"{sa_feature}_{axis}" for sa_feature in sa_features.keys() for axis in ['x', 'y', 'z']]
    feature_df = pd.DataFrame(data=accel_features, columns=columns)
    return feature_df

## Sit2Stand

In [7]:
def get_sit2stand(data):
    sit2stand = Sit2Stand(stillness_constraint=False) # setting to False is recommended for clinical data
    s2s_results = sit2stand.predict(time=data['time'], accel=data['accel'])
    s2s_n = len(s2s_results['Max. Accel.'])
    s2s_duration = np.mean(s2s_results['Duration']) if s2s_n >= 5 else None
    s2s_max_accel = np.mean(s2s_results['Max. Accel.']) if s2s_n >= 5 else None
    s2s_min_accel = np.mean(s2s_results['Min. Accel.']) if s2s_n >= 5 else None
    s2s_SPARC = np.mean(s2s_results['SPARC']) if s2s_n >= 5 else None
    s2s_displacement = np.mean(s2s_results['Vertical Displacement']) if s2s_n >= 5 else None
    s2s = np.array([[s2s_n, s2s_duration, s2s_max_accel, s2s_min_accel, s2s_SPARC, s2s_displacement]])
    s2s_df = pd.DataFrame(data=s2s, columns=['s2s_n', 's2s_duration', 's2s_max_accel',  's2s_min_accel',  's2s_SPARC', 's2s_displacement'])
    return s2s_df

# Mobgap

## Preprocessing

In [8]:
def make_df(sdkh_data):
    data = sdkh_data.copy()
    for i, axis in enumerate(['x', 'y', 'z']):
        data[f'acc_{axis}'] = np.array([row[i] for row in data['accel']])
        data[f'gyr_{axis}'] = np.array([row[i] for row in data['gyro']])
    for axis in ['acc_x', 'acc_y', 'acc_z']:
        data[axis] = data[axis]*9.80665 # Conversion from g to m/s²
    data.pop('accel')
    data.pop('gyro')
    data = pd.DataFrame(data)
    data['datetime'] = pd.to_datetime(data['time'], unit='s').dt.floor('s')
    return data

## Pipeline

In [9]:
def run_mobgap_analysis(imu_data, sampling_rate_hz, participant_metadata):
    gsd = GsdIluz()
    icd = IcdShinImproved()
    lrc = LrcUllrich()
    cad = CadFromIc()
    sl = SlZijlstra()
    speed = WsNaive()
    turn = TdElGohary()
    gs_iterator = GsIterator()
    
    gsd.detect(imu_data, sampling_rate_hz=sampling_rate_hz, **participant_metadata)
    gait_sequences = gsd.gs_list_

    for (_, gs_data), r in gs_iterator.iterate(imu_data, gait_sequences):
        icd = icd.clone().detect(gs_data, sampling_rate_hz=sampling_rate_hz, **participant_metadata)
        lrc = lrc.clone().predict(gs_data, icd.ic_list_, sampling_rate_hz=sampling_rate_hz)
        r.ic_list = lrc.ic_lr_list_
        turn = turn.clone().detect(gs_data, sampling_rate_hz=sampling_rate_hz)
        r.turn_list = turn.turn_list_
    
        refined_gs, refined_ic_list = refine_gs(r.ic_list)
    
        with gs_iterator.subregion(refined_gs) as ((_, refined_gs_data), rr):
            cad = cad.clone().calculate(
                refined_gs_data,
                initial_contacts=refined_ic_list,
                sampling_rate_hz=sampling_rate_hz,
                **participant_metadata,
            )
            rr.cadence_per_sec = cad.cadence_per_sec_
            sl = sl.clone().calculate(
                refined_gs_data,
                initial_contacts=refined_ic_list,
                sampling_rate_hz=sampling_rate_hz,
                **participant_metadata,
            )
            rr.stride_length_per_sec = sl.stride_length_per_sec_
            speed = speed.clone().calculate(
                refined_gs_data,
                initial_contacts=refined_ic_list,
                cadence_per_sec=cad.cadence_per_sec_,
                stride_length_per_sec=sl.stride_length_per_sec_,
                sampling_rate_hz=sampling_rate_hz,
                **participant_metadata,
            )
            rr.walking_speed_per_sec = speed.walking_speed_per_sec_
        
    return gs_iterator.results_, gait_sequences

In [10]:
def process_mobgap_results(results, sampling_rate_hz, participant_metadata):
    
    if results.cadence_per_sec.empty:
        return pd.DataFrame()
        
    combined_results = pd.concat(
        [
            results.cadence_per_sec,
            results.stride_length_per_sec,
            results.walking_speed_per_sec,
        ],
        axis=1,
    ).reset_index("r_gs_id", drop=True)
    
    stride_list = (
        results.ic_list.groupby("gs_id", group_keys=False)
        .apply(strides_list_from_ic_lr_list)
        .assign(stride_duration_s=lambda df_: (df_.end - df_.start) / sampling_rate_hz)
    )
    
    stride_list_with_approx_paras = create_multi_groupby(
        primary_df=stride_list,
        secondary_dfs=combined_results,
        groupby="gs_id",
        group_keys=False,
    ).apply(naive_sec_paras_to_regions, sampling_rate_hz=sampling_rate_hz)
    
    flat_index = pd.Index(
        [
            "_".join(str(e) for e in s_id)
            for s_id in stride_list_with_approx_paras.index
        ],
        name="s_id",
    )
    
    stride_list_with_approx_paras = (
        stride_list_with_approx_paras.reset_index("gs_id")
        .rename(columns={"gs_id": "original_gs_id"})
        .set_index(flat_index)
    )

    ss = StrideSelection().filter(stride_list_with_approx_paras, sampling_rate_hz=sampling_rate_hz)
    wba = WbAssembly().assemble(ss.filtered_stride_list_, sampling_rate_hz=sampling_rate_hz)
    
    final_strides = wba.annotated_stride_list_
    per_wb_params = wba.wb_meta_parameters_ 
    params_to_aggregate = [
        "stride_duration_s",
        "cadence_spm",
        "stride_length_m",
        "walking_speed_mps",
    ]
    
    per_wb_params = pd.concat(
        [
            per_wb_params,
            final_strides.reindex(columns=params_to_aggregate)
            .groupby(["wb_id"])
            .mean(),
        ],
        axis=1,
    )

    per_wb_params_mask = apply_thresholds(
        input_data=per_wb_params,
        thresholds=get_mobilised_dmo_thresholds(),
        cohort='PFF', # irrelevant if measurement_condition='global', else cohort restricted to proximal femur fractures
        height_m=participant_metadata["height_m"],
        measurement_condition='global', # alternative: 'free-living'
    )

    agg = MobilisedAggregator(**MobilisedAggregator.PredefinedParameters.single_recording)
    agg_results = agg.aggregate(per_wb_params, wb_dmos_mask=per_wb_params_mask).aggregated_data_

    return agg_results.reset_index(drop=True)

# Analysis

In [11]:
results_df = pd.DataFrame()

for index, subject in catalog.iterrows():

    # if index < 47:  # If the calculation crashes you can use this code to pick up where it left off
    #     continue

    print()
    print(f'Subject: {subject['ID']}')

    # Loading sample metadata
    participant_metadata = {
        'cohort': 'SA',
        'height_m': subject['height_m'],
        'indip_data_used': 'All',
        'sensor_attachment_su': subject['sensor_attachment_su'],
        'sensor_height_m': subject['sensor_height_m'],
        'sensor_type_su': 'AX6',
        'weight_kg': subject['weight_kg']
    }

    # Loading Sample
    print('Loading data...')
    reader = ReadCwa()
    with warnings.catch_warnings(): # Ignore Timezone Warning
        warnings.simplefilter("ignore", category=UserWarning)
        skdh_data = reader.predict(file=f'./01_Rohdaten/{subject['file']}')
    sampling_rate_hz = skdh_data['fs']

    # Check worn interval
    start, end = wear_detection(skdh_data)
    if start > end:
        print('Sensor worn for less than an hour. No analysis possible.')
        continue
        
    # Slicing data to wearing episode
    sensor_worn_s = (end - start) / 100
    worn_df = pd.DataFrame(data=np.array([[subject['ID']]]), columns=['ID'])
    worn_df['sensor_worn_s'] = sensor_worn_s
    data_worn = slice_sample(skdh_data, start, end)

    # Get sensor features
    print('Calculating signal features...')
    feature_df = get_features(data_worn)

    if participant_metadata['sensor_attachment_su'] == 'lumbar':
        # Get Sit2Stand data
        print('Calculating Sit2Stand...')
        s2s_df = get_sit2stand(data_worn)
        
        # Preprocessing data for mobgap
        mobgap_data = make_df(skdh_data)
        imu_data = to_body_frame(mobgap_data)
    
        # Calculate walking bouts
        print('Run mobgap analysis...')
        mobgap_results, gait_sequences = run_mobgap_analysis(imu_data, sampling_rate_hz, participant_metadata)
    
        # Aggregate data
        print('Aggregate results...')
        mobgap_df = process_mobgap_results(mobgap_results, sampling_rate_hz, participant_metadata)

        # Diary analysis
        if subject['diary'] == True:
            gait_sequences['start'] = gait_sequences['start'].apply(lambda x: mobgap_data['datetime'][x])
            gait_sequences['end'] = gait_sequences['end'].apply(lambda x: mobgap_data['datetime'][x])
            gait_sequences.to_csv(f'./03_Ergebnisse/{subject['ID']}/gait_sequences.csv')
    
    else:
        s2s_df = pd.DataFrame()
        mobgap_df = pd.DataFrame()
        
    # Concat and save sample results
    sample_results = pd.concat([worn_df, feature_df, s2s_df, mobgap_df], axis=1)
    sample_results.to_csv(f'./03_Ergebnisse/{subject['ID']}/results.csv')
    
    # Expand results_df
    results_df = pd.concat([results_df, sample_results], axis=0, ignore_index=True)



Subject: 11005
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 11007
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 11014
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 11017
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 11021
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 11022
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 11024
Loading data...
Sensor worn for less than an hour. No analysis possible.

Subject: 11026
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregat

/home/thomas.kocar/Devel/sklearn/env/lib/python3.12/site-packages/skdh/io/axivity.py:110: RuntimeWarning: Bad data blocks present
  fs, n_bad_samples, imudata, ts, temperature = read_axivity(str(file))


Calculating signal features...

Subject: 11058
Loading data...
Calculating signal features...

Subject: 11062
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 11065
Loading data...
Calculating signal features...
Calculating Sit2Stand...


/home/thomas.kocar/Devel/sklearn/env/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thomas.kocar/Devel/sklearn/env/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Run mobgap analysis...
Aggregate results...

Subject: 11066
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 11069
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 11074
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 11078
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 11083
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 11085
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 11088
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 11091
Loading data...


/home/thomas.kocar/Devel/sklearn/env/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/thomas.kocar/Devel/sklearn/env/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Run mobgap analysis...
Aggregate results...

Subject: 12006
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 12014
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 13004
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 13005
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 13008
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 13014
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 13016
Loading data...
Calculating signal features...
Calculating Sit2Stand...
Run mobgap analysis...
Aggregate results...

Subject: 13017
Loading data...


In [12]:
results_df

,ID,sensor_worn_s,mean_x,mean_y,mean_z,standard_deviation_x,standard_deviation_y,standard_deviation_z,skewness_x,skewness_y,...,wb_30__count,wb_30__walking_speed_mps__avg,wb_30__stride_length_m__avg,wb_30__cadence_spm__avg,wb_30__stride_duration_s__avg,wb_30__walking_speed_mps__max,wb_30__cadence_spm__max,wb_30__walking_speed_mps__var,wb_30__stride_length_m__var,wb_60__count
0,11005,104192.00,0.237570,-0.165078,0.970104,0.176409,0.070960,0.162409,2.926927,0.678078,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,11007,297293.00,0.240609,-0.050715,0.924362,0.231565,0.299405,0.189608,1.486305,-0.977817,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,11014,368141.00,0.035221,0.228935,0.908813,0.151329,0.207512,0.234599,1.765815,1.812948,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,11017,160906.00,0.400680,0.098482,0.918323,0.158952,0.201004,0.132436,0.278038,-0.004607,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,11021,294990.00,0.459312,0.196493,0.494586,0.318285,0.401947,0.515721,0.864517,-0.024539,...,2,0.524883,0.922961,68.134926,1.114074,0.541308,69.150750,0.055317,0.027273,0
5,11022,422816.79,0.304249,-0.225882,0.955560,0.155295,0.113813,0.081933,0.321099,-0.376244,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
6,11026,433575.19,0.385920,-0.238025,0.700568,0.321579,0.254086,0.408761,0.957984,1.973155,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,11030,364155.00,0.325058,0.244076,0.756785,0.256530,0.285667,0.361010,1.831196,-0.650035,...,3,0.918454,1.039152,103.948590,0.838572,1.083815,111.781120,0.298844,0.195630,0
8,11031,365223.00,0.387914,-0.041547,0.575990,0.302110,0.490330,0.484839,1.257858,-0.438518,...,5,0.527810,0.672877,94.094588,1.269699,0.548526,95.260987,0.041229,0.036482,1
9,11036,435397.19,-0.011015,-0.138765,-0.950336,0.150631,0.183249,0.100702,0.201690,-1.319937,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>


In [13]:
results_df.to_csv('./results.csv')